In [1]:
%matplotlib inline
import numpy as np
import os, sys, random

paths = {}
paths['dir'] = os.path.abspath(os.path.dirname(os.path.realpath('__file__')))
paths['TestFile'] = os.path.join(paths['dir'],'program.lpf')

sys.path.append(paths['dir'])
import LPFParser as lpfp
import LPFEncoder as lpfe

In [ ]:
# General parameters
while True:
    try:
        rowNum = int(input("Enter the number of rows in the plate: "))
        colNum = int(input("Enter the number of columns in the plate: "))
        totalTime = int(input("Enter the total time for the experiment in minutes: "))
        ledsPerWell = int(input("Enter the number of LEDs per well: "))
        if rowNum > 0 and colNum > 0 and totalTime > 0 and ledsPerWell > 0:
            break
        else:
            print("All values must be positive integers. Please enter again.")
    except ValueError:
        print("Invalid input. Please enter positive integers.")
# Device params:
timeStep = 10  # ms
# Compute time array
times = np.arange(0, totalTime * 60 * 1000 + timeStep, timeStep)
dp = {}
dp['channelNum'] = rowNum * colNum * ledsPerWell # TOTAL number of channels
dp['numSteps'] = len(times)
dp['timeStep'] = timeStep


# Initialize LPA_mat
LPA_mat = np.zeros((len(times), rowNum, colNum, ledsPerWell), dtype=np.int16)

# Loop through each well and collect input
for row in range(rowNum):
    for col in range(colNum):
        print(f"\nConfiguring well ({row + 1}, {col + 1}):")

        # Selected LED
        while True:
            try:
                selected_led = int(input(f"Enter the LED index (0-{ledsPerWell - 1}): "))
                if 0 <= selected_led < ledsPerWell:
                    break
                else:
                    print(f"Please enter a valid LED index between 0 and {ledsPerWell - 1}.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        # Frequency in Hz
        while True:
            try:
                frequency = float(input(f"Enter the frequency in Hz for well ({row + 1}, {col + 1}): "))
                if frequency > 0:
                    break
                else:
                    print("Frequency must be a positive number.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        # High intensity percentage
        while True:
            try:
                high_intensity = int(input(f"Enter the high intensity (percentage of maximum) for well ({row + 1}, {col + 1}): "))
                if 0 <= high_intensity <= 100:
                    break
                else:
                    print("High intensity must be between 0 and 100.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        # Low intensity percentage
        while True:
            try:
                low_intensity = int(input(f"Enter the low intensity (percentage of maximum) for well ({row + 1}, {col + 1}): "))
                if 0 <= low_intensity <= high_intensity:
                    break
                else:
                    print("Low intensity must be between 0 and High intensity.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        # Illumination duration in ms
        while True:
            try:
                illumination_dur = int(input(f"Enter the illumination duration in milliseconds for well ({row + 1}, {col + 1}): "))
                if 0 < illumination_dur <= totalTime * 60 * 1000:
                    break
                else:
                    print(f"Duration must be between 0 and {totalTime * 60 * 1000} ms.")
            except ValueError:
                print("Invalid input. Please enter an integer.")

        # Compute period and duration for this well
        period = int(1 / frequency * 1000)  # Convert frequency to period in ms

        for ti, t in enumerate(times):  # Iterate through all timesteps
            if (t % period) < illumination_dur:
                LPA_mat[ti, row, col, selected_led] = int(high_intensity / 100 * 4095)
            else:
                LPA_mat[ti, row, col, selected_led] = int(low_intensity / 100 * 4095)

# Finalize by printing the shape of the generated matrix
print(f"\nLPA_mat generated with shape {LPA_mat.shape}.")
print("Configuration complete.")

In [18]:
import numpy as np

np.savetxt("LPA_mat.csv", LPA_mat.reshape(-1, LPA_mat.shape[-1]), delimiter=",", fmt='%d')
print("Matrix saved to 'LPA_mat.csv'")


Matrix saved to 'LPA_mat.csv'


In [20]:
print(LPA_mat)

[[[[3685    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[3685    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[3685    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]
   [   0 3276]]]


 [[[ 409    0]
   [   0  204]]]


 [[[ 409    0]

In [ ]:
lpfe.LPFEncoder(LPA_mat, dp, paths['TestFile'])

In [ ]:
lpfdata = lpfp.LPFtoArray(paths['TestFile'],rowNum, colNum,ledsPerWell) # Can take some time

In [ ]:
## Plot the 100ms period (well "0")
wellIndices = [0] # Just the first well
#wellIndices = lpfp.trueIndex(wellIndices, rm) # derandomize
mplargs={'xlim': (0,0.01),
         'chLabels': ['Red', 'Green'],
         'ylim': (-800,4200)}
lpfp.plotLPFData(lpfdata['data'], wellIndices=wellIndices, mplargs=mplargs)